In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/penyisihan-datavidia-10/sample_submission.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2012-komponen-data.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2010-komponen-data.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2018-komponen-data.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2025.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2014-komponen-data.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2024.csv
/kaggle/input/penyisihan-datavidia-10/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2011-kompone

# Data Loading & Sanity Checks

In [2]:
# ============================================================
# STAGE 1 — Data Loading & Sanity Checks (ISPU 2010–2025) — ONE CELL (REVISI FINAL)
# Tujuan:
# - Load semua file ISPU (2010–2025) dari folder /ISPU
# - Standarisasi kolom (tanggal, stasiun_code, polutan, max, kategori)
# - Labeling sesuai FAQ panitia -> 3 kelas: BAIK / SEDANG / TIDAK SEHAT
# - Buang stasiun tidak valid (hanya DKI1..DKI5)
# - Deduplicate (tanggal, stasiun_code) dengan agregasi aman
# Output penting untuk stage berikutnya:
# - df_ispu  (daily clean)
# - CUTOFF_DATE, DATA_ROOT, VALID_STATIONS
# ============================================================

from pathlib import Path
import re
import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)

DATA_ROOT = Path("/kaggle/input/penyisihan-datavidia-10")
ISPU_DIR  = DATA_ROOT / "ISPU"
CUTOFF_DATE = pd.Timestamp("2025-09-01")  # POV sebelum tanggal ini (aturan panitia)

VALID_STATIONS = ["DKI1", "DKI2", "DKI3", "DKI4", "DKI5"]

# ---------- helpers ----------
def norm_colname(c: str) -> str:
    c = str(c).strip().lower()
    c = re.sub(r"[()\[\]{}]", "", c)
    c = re.sub(r"[%/]", "_", c)
    c = re.sub(r"[^a-z0-9]+", "_", c)
    c = re.sub(r"_+", "_", c).strip("_")
    return c

def read_csv_try_enc(p: Path) -> pd.DataFrame:
    # sederhana tapi robust untuk encoding berbeda
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(p)  # last resort

def coalesce_numeric(df: pd.DataFrame, candidates):
    cols = [c for c in candidates if c in df.columns]
    if len(cols) == 0:
        return pd.Series(np.nan, index=df.index)
    tmp = df[cols].apply(pd.to_numeric, errors="coerce")
    return tmp.mean(axis=1, skipna=True)

def coalesce_text(df: pd.DataFrame, candidates):
    cols = [c for c in candidates if c in df.columns]
    if len(cols) == 0:
        return pd.Series(np.nan, index=df.index)
    tmp = df[cols].astype(str)
    tmp = tmp.replace({"nan": np.nan, "None": np.nan, "": np.nan})
    return tmp.bfill(axis=1).iloc[:, 0]

def mode_nonnull(x: pd.Series):
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    m = x.mode()
    return m.iloc[0] if len(m) else np.nan

def extract_stasiun_code(stasiun_series: pd.Series) -> pd.Series:
    s = stasiun_series.astype(str).str.upper()
    code = s.str.extract(r"(DKI\s*\d+)", expand=False)
    code = code.str.replace(" ", "", regex=False)
    return code  # bisa NaN kalau tidak match

# ---------- column synonym sets (hindari duplikat nama kolom) ----------
DATE_CANDS = ["tanggal", "time", "date", "waktu", "tgl", "tanggal_pengukuran"]
STA_CANDS  = ["stasiun", "stasiun_id", "station", "lokasi", "nama_stasiun"]

COLS_NUM = {
    "pm_sepuluh":        ["pm_sepuluh","pm10","pm_10","pm10_","pm_10_"],
    "pm_duakomalima":    ["pm_duakomalima","pm25","pm2_5","pm_2_5","pm2.5","pm_2.5"],
    "sulfur_dioksida":   ["sulfur_dioksida","so2"],
    "karbon_monoksida":  ["karbon_monoksida","co"],
    "ozon":              ["ozon","o3"],
    "nitrogen_dioksida": ["nitrogen_dioksida","no2"],
    "max":               ["max","maks","nilai_maks","nilai_max"],
}

COLS_TXT = {
    "parameter_pencemar_kritis": ["parameter_pencemar_kritis","parameter_kritis","parameter","pencemar_kritis"],
    "kategori_raw":              ["kategori","category","kategori_ispu","kategori_ispa","status"],
}

# ---------- load all ISPU files ----------
ispu_files = sorted(ISPU_DIR.glob("*.csv"))
if len(ispu_files) == 0:
    raise FileNotFoundError(f"Tidak menemukan file ISPU di: {ISPU_DIR}")

rows = []
file_stats = []

for p in ispu_files:
    df0 = read_csv_try_enc(p)
    df0.columns = [norm_colname(c) for c in df0.columns]
    df0["__source_file"] = p.name

    # tanggal & stasiun (ambil dari kandidat yang tersedia)
    tanggal = coalesce_text(df0, DATE_CANDS)
    stasiun = coalesce_text(df0, STA_CANDS)

    df1 = pd.DataFrame({
        "tanggal": pd.to_datetime(tanggal, errors="coerce", dayfirst=True),
        "stasiun": stasiun,
        "__source_file": df0["__source_file"].values,
    })

    # numeric pollutants
    for k, cands in COLS_NUM.items():
        df1[k] = coalesce_numeric(df0, cands)

    # text
    for k, cands in COLS_TXT.items():
        df1[k] = coalesce_text(df0, cands)

    # station code
    df1["stasiun_code"] = extract_stasiun_code(df1["stasiun"])

    # quick file parse stats
    parsed_rate = float(df1["tanggal"].notna().mean())
    max_dt = df1["tanggal"].max()
    file_stats.append((p.name, len(df1), parsed_rate, max_dt))

    rows.append(df1)

df_raw = pd.concat(rows, ignore_index=True)

# ---------- basic validity ----------
df_raw = df_raw.loc[df_raw["tanggal"].notna()].copy()
df_raw["stasiun_code"] = df_raw["stasiun_code"].astype("string")

# ---------- map kategori -> 3 classes ----------
cat = df_raw["kategori_raw"].astype("string").str.upper().str.strip()
cat = cat.str.replace(r"\s+", " ", regex=True)

cat = cat.replace({
    "SANGAT TIDAK SEHAT": "TIDAK SEHAT",
    "BERBAHAYA": "TIDAK SEHAT",
    "TIDAKSEHAT": "TIDAK SEHAT",
    "TIDAK  SEHAT": "TIDAK SEHAT",
})
df_raw["kategori_3"] = cat.where(cat.isin(["BAIK", "SEDANG", "TIDAK SEHAT"]), np.nan)

# ---------- keep only valid stations (drop NAN / unknown) ----------
df_raw_valid = df_raw.loc[df_raw["stasiun_code"].isin(VALID_STATIONS)].copy()

# ---------- deduplicate per (tanggal, stasiun_code) ----------
num_cols = list(COLS_NUM.keys())
group_keys = ["tanggal", "stasiun_code"]

agg = {c: "mean" for c in num_cols}
agg.update({
    "kategori_3": mode_nonnull,
    "stasiun": mode_nonnull,
    "parameter_pencemar_kritis": mode_nonnull,
    "__source_file": mode_nonnull,
})

df_ispu = (
    df_raw_valid.groupby(group_keys, as_index=False)
    .agg(agg)
    .sort_values(group_keys)
    .reset_index(drop=True)
)

# ---------- summaries ----------
min_date = df_ispu["tanggal"].min()
max_date = df_ispu["tanggal"].max()

print("=== STAGE 1 SUMMARY (ISPU) ===")
print(f"Files loaded: {len(ispu_files)}")
print(f"Rows raw (all stations): {len(df_raw):,}")
print(f"Rows valid stations (pre-agg): {len(df_raw_valid):,}")
print(f"Rows daily (after agg): {len(df_ispu):,}")
print(f"Stations (valid): {df_ispu['stasiun_code'].nunique()} -> {sorted(df_ispu['stasiun_code'].unique().tolist())}")
print(f"Date range: {min_date.date()} -> {max_date.date()}")
print(f"CUTOFF_DATE (forecasting POV): {CUTOFF_DATE.date()}")

print("\nLabel counts (kategori_3) on daily table:")
print(df_ispu["kategori_3"].value_counts(dropna=False))

miss = df_ispu[num_cols + ["kategori_3"]].isna().mean().sort_values(ascending=False)
print("\nMissing rate (top):")
print(miss.head(15))

per_sta = df_ispu.groupby("stasiun_code").agg(
    n_days=("tanggal", "count"),
    min_date=("tanggal", "min"),
    max_date=("tanggal", "max"),
    label_na_rate=("kategori_3", lambda s: float(s.isna().mean()))
).sort_values("n_days", ascending=False)
print("\nPer-station coverage:")
print(per_sta)

# per-file date parse diagnostics (untuk cek kenapa 2024/2025 bisa gagal)
df_file_stats = pd.DataFrame(file_stats, columns=["file","n_rows","date_parse_rate","max_date_parsed"])
df_file_stats = df_file_stats.sort_values(["date_parse_rate","max_date_parsed"], ascending=[True, True])
print("\nPer-file date parsing diagnostics (worst -> best):")
print(df_file_stats.head(10))

# handy globals for next stages
ISPU_MIN_DATE = min_date
ISPU_MAX_DATE = max_date


/tmp/ipykernel_17/255431024.py:111: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  "tanggal": pd.to_datetime(tanggal, errors="coerce", dayfirst=True),
/tmp/ipykernel_17/255431024.py:111: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  "tanggal": pd.to_datetime(tanggal, errors="coerce", dayfirst=True),
/tmp/ipykernel_17/255431024.py:111: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  "tanggal": pd.to_datetime(tanggal, errors="coerce", dayfirst=True),
/tmp/ipykernel_17/255431024.py:111: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silenc

=== STAGE 1 SUMMARY (ISPU) ===
Files loaded: 16
Rows raw (all stations): 10,541
Rows valid stations (pre-agg): 8,740
Rows daily (after agg): 8,637
Stations (valid): 5 -> ['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']
Date range: 2010-01-01 -> 2023-11-30
CUTOFF_DATE (forecasting POV): 2025-09-01

Label counts (kategori_3) on daily table:
kategori_3
<NA>           6833
SEDANG         1358
BAIK            236
TIDAK SEHAT     210
Name: count, dtype: Int64

Missing rate (top):
kategori_3           0.791131
pm_duakomalima       0.623133
pm_sepuluh           0.211416
sulfur_dioksida      0.185365
ozon                 0.182471
nitrogen_dioksida    0.182239
karbon_monoksida     0.178534
max                  0.001042
dtype: float64

Per-station coverage:
              n_days   min_date   max_date  label_na_rate
stasiun_code                                             
DKI1            1745 2010-01-01 2023-11-30       0.793123
DKI2            1723 2010-01-01 2023-11-30       0.789321
DKI3            172

# Master Table Building (Correct Joins)

In [3]:
# ============================================================
# STAGE 2 — Master Table Building (Correct Joins) — ONE CELL
# Tujuan:
# - Parse sample_submission -> (id, tanggal_target, stasiun_code)
# - Build df_targets (forecast frame) + join kalender & libur (allowed)
# - Compute last_obs_date per stasiun from df_ispu + horizon_days
# - Load aux tables (NDVI, cuaca, air sungai, penduduk) clean minimal (dipakai stage berikutnya)
# Output utama:
# - df_sub, df_calendar, df_targets
# - df_ispu_hist, last_obs_by_station
# - df_ndvi, df_weather, df_water_m, df_pop_y
# ============================================================

from pathlib import Path
import re
import numpy as np
import pandas as pd

# ---------- guards ----------
need = ["df_ispu", "DATA_ROOT", "CUTOFF_DATE", "VALID_STATIONS"]
miss = [k for k in need if k not in globals()]
if miss:
    raise RuntimeError(f"Missing globals from STAGE 1: {miss}")

SUB_PATH   = DATA_ROOT / "sample_submission.csv"
HOL_PATH   = DATA_ROOT / "libur-nasional" / "dataset-libur-nasional-dan-weekend.csv"
NDVI_PATH  = DATA_ROOT / "NDVI (vegetation index)" / "indeks-ndvi-jakarta.csv"
WATER_PATH = DATA_ROOT / "kualitas-air-sungai" / "data-kualitas-air-sungai-komponen-data.csv"
POP_PATH   = DATA_ROOT / "jumlah-penduduk" / "data-jumlah-penduduk-provinsi-dki-jakarta-berdasarkan-kelompok-usia-dan-jenis-kelamin-tahun-2013-2021-komponen-data.csv"
WEATHER_DIR = DATA_ROOT / "cuaca-harian"

def norm_colname(c: str) -> str:
    c = str(c).strip().lower()
    c = re.sub(r"[()\[\]{}]", "", c)
    c = re.sub(r"[%/]", "_", c)
    c = re.sub(r"[^a-z0-9]+", "_", c)
    c = re.sub(r"_+", "_", c).strip("_")
    return c

def extract_station(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.upper()
    code = x.str.extract(r"(DKI\s*\d+)", expand=False)
    return code.str.replace(" ", "", regex=False)

def extract_date(s: pd.Series) -> pd.Series:
    x = s.astype(str)
    tok = x.str.extract(r"(\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{4})", expand=False)
    return pd.to_datetime(tok, errors="coerce", dayfirst=True)

# ============================================================
# 2.1 Parse sample_submission -> df_sub
# ============================================================
df_sub0 = pd.read_csv(SUB_PATH)
df_sub0.columns = [norm_colname(c) for c in df_sub0.columns]
id_col = "id" if "id" in df_sub0.columns else df_sub0.columns[0]

df_sub = pd.DataFrame({
    "id": df_sub0[id_col].astype(str),
})
df_sub["tanggal_target"] = extract_date(df_sub["id"])
df_sub["stasiun_code"] = extract_station(df_sub["id"])

df_sub = df_sub.loc[df_sub["tanggal_target"].notna() & df_sub["stasiun_code"].notna()].copy()
df_sub = df_sub.loc[df_sub["stasiun_code"].isin(VALID_STATIONS)].reset_index(drop=True)

# ============================================================
# 2.2 ISPU history (untuk last_obs_date) — batasi sebelum CUTOFF_DATE
# ============================================================
df_ispu_hist = df_ispu.loc[
    df_ispu["stasiun_code"].isin(VALID_STATIONS) &
    (df_ispu["tanggal"] < CUTOFF_DATE)
].copy()

last_obs_by_station = df_ispu_hist.groupby("stasiun_code")["tanggal"].max()

# ============================================================
# 2.3 Calendar table + holiday (allowed future features)
# ============================================================
cal_min = min(df_ispu_hist["tanggal"].min(), df_sub["tanggal_target"].min())
cal_max = max(df_ispu_hist["tanggal"].max(), df_sub["tanggal_target"].max())

df_calendar = pd.DataFrame({"tanggal": pd.date_range(cal_min, cal_max, freq="D")})
df_calendar["year"] = df_calendar["tanggal"].dt.year
df_calendar["month"] = df_calendar["tanggal"].dt.month
df_calendar["day"] = df_calendar["tanggal"].dt.day
df_calendar["dow"] = df_calendar["tanggal"].dt.dayofweek
df_calendar["dayofyear"] = df_calendar["tanggal"].dt.dayofyear

df_calendar["sin_doy"] = np.sin(2*np.pi*df_calendar["dayofyear"]/365.25)
df_calendar["cos_doy"] = np.cos(2*np.pi*df_calendar["dayofyear"]/365.25)
df_calendar["sin_month"] = np.sin(2*np.pi*df_calendar["month"]/12.0)
df_calendar["cos_month"] = np.cos(2*np.pi*df_calendar["month"]/12.0)

df_hol = pd.read_csv(HOL_PATH)
df_hol.columns = [norm_colname(c) for c in df_hol.columns]
df_hol["tanggal"] = pd.to_datetime(df_hol["tanggal"], errors="coerce", dayfirst=True)

keep_h = [c for c in ["tanggal","is_holiday_nasional","nama_libur","is_weekend","day_name"] if c in df_hol.columns]
df_hol = df_hol[keep_h].dropna(subset=["tanggal"]).drop_duplicates("tanggal")

df_calendar = df_calendar.merge(df_hol, on="tanggal", how="left")

# fallback weekend/holiday values
df_calendar["is_weekend"] = df_calendar["is_weekend"].fillna((df_calendar["dow"] >= 5).astype(int)).astype(int)
df_calendar["is_holiday_nasional"] = df_calendar["is_holiday_nasional"].fillna(0).astype(int)

# ============================================================
# 2.4 df_targets = submission index + kalender/libur + last_obs/horizon
# ============================================================
df_targets = df_sub.merge(
    df_calendar,
    left_on="tanggal_target",
    right_on="tanggal",
    how="left"
).drop(columns=["tanggal"])

df_targets["last_obs_date"] = df_targets["stasiun_code"].map(last_obs_by_station)
df_targets["horizon_days"] = (df_targets["tanggal_target"] - df_targets["last_obs_date"]).dt.days

# ============================================================
# 2.5 Load auxiliary tables (clean minimal; dipakai stage berikutnya)
# ============================================================

# NDVI
df_ndvi = pd.read_csv(NDVI_PATH)
df_ndvi.columns = [norm_colname(c) for c in df_ndvi.columns]
if "tanggal" in df_ndvi.columns:
    df_ndvi["tanggal"] = pd.to_datetime(df_ndvi["tanggal"], errors="coerce", dayfirst=True)

# map station id -> stasiun_code
if "stasiun_code" not in df_ndvi.columns:
    sid = df_ndvi["stasiun_id"] if "stasiun_id" in df_ndvi.columns else (df_ndvi["stasiun"] if "stasiun" in df_ndvi.columns else pd.Series("", index=df_ndvi.index))
    df_ndvi["stasiun_code"] = sid.astype(str).str.upper().str.replace(" ", "", regex=False)

df_ndvi = df_ndvi.loc[df_ndvi["stasiun_code"].isin(VALID_STATIONS)].copy()

# Weather (5 file DKI1..DKI5)
w_files = sorted(WEATHER_DIR.glob("cuaca-harian-*.csv"))
w_list = []
for p in w_files:
    m = re.search(r"dki(\d+)", p.name.lower())
    st_code = f"DKI{m.group(1)}" if m else None
    if st_code not in VALID_STATIONS:
        continue
    d = pd.read_csv(p)
    d.columns = [norm_colname(c) for c in d.columns]
    tcol = [c for c in ["time","tanggal","date","waktu"] if c in d.columns][0]
    d["tanggal"] = pd.to_datetime(d[tcol], errors="coerce", dayfirst=True)
    d["stasiun_code"] = st_code
    d = d.dropna(subset=["tanggal"]).drop(columns=[tcol], errors="ignore")
    w_list.append(d)

df_weather = pd.concat(w_list, ignore_index=True) if len(w_list) else pd.DataFrame(columns=["tanggal","stasiun_code"])

# Water quality -> aggregate monthly
df_water = pd.read_csv(WATER_PATH)
df_water.columns = [norm_colname(c) for c in df_water.columns]

for c in ["hasil_pengukuran","baku_mutu"]:
    if c in df_water.columns:
        df_water[c] = pd.to_numeric(df_water[c], errors="coerce")

# derive year/month
if "periode_data" in df_water.columns:
    df_water["tahun"] = pd.to_numeric(df_water["periode_data"], errors="coerce")
else:
    df_water["tahun"] = np.nan
if "bulan_sampling" in df_water.columns:
    df_water["bulan"] = pd.to_numeric(df_water["bulan_sampling"], errors="coerce")
else:
    df_water["bulan"] = np.nan

df_water["exceed_ratio"] = df_water["hasil_pengukuran"] / df_water["baku_mutu"]

df_water_m = (
    df_water.dropna(subset=["tahun","bulan"])
           .groupby(["tahun","bulan"], as_index=False)
           .agg(
               water_exceed_mean=("exceed_ratio","mean"),
               water_exceed_rate=("exceed_ratio", lambda s: float((s > 1).mean())),
               water_n=("exceed_ratio","count")
           )
)

# Population -> yearly total + yoy
df_pop = pd.read_csv(POP_PATH)
df_pop.columns = [norm_colname(c) for c in df_pop.columns]
year_col = "tahun" if "tahun" in df_pop.columns else ("periode_data" if "periode_data" in df_pop.columns else None)
df_pop["tahun"] = pd.to_numeric(df_pop[year_col], errors="coerce") if year_col else np.nan
df_pop["jumlah_penduduk"] = pd.to_numeric(df_pop["jumlah_penduduk"], errors="coerce") if "jumlah_penduduk" in df_pop.columns else np.nan

df_pop_y = (
    df_pop.dropna(subset=["tahun"])
          .groupby("tahun", as_index=False)
          .agg(pop_total=("jumlah_penduduk","sum"))
          .sort_values("tahun")
)
df_pop_y["pop_yoy"] = df_pop_y["pop_total"].pct_change()

# ============================================================
# 2.6 QA prints
# ============================================================
print("=== STAGE 2 SUMMARY ===")
print("Submission rows:", len(df_sub))
print("Target date range:", df_sub["tanggal_target"].min().date(), "->", df_sub["tanggal_target"].max().date())
print("Stations in submission:", sorted(df_sub["stasiun_code"].unique().tolist()))

print("\nLast observed date per station (from ISPU hist):")
print(last_obs_by_station)

print("\nHorizon_days summary:")
print(df_targets["horizon_days"].describe())

print("\ndf_targets preview:")
print(df_targets[["id","tanggal_target","stasiun_code","last_obs_date","horizon_days","is_weekend","is_holiday_nasional"]].head(10))

# warning if ISPU history ends far before cutoff (affects forecasting difficulty)
hist_max = df_ispu_hist["tanggal"].max()
gap_days = int((CUTOFF_DATE - hist_max).days) if pd.notna(hist_max) else None
print("\nISPU hist max date:", hist_max.date() if pd.notna(hist_max) else hist_max, "| gap to CUTOFF_DATE (days):", gap_days)

print("\nAux tables shapes:")
print("NDVI:", df_ndvi.shape, "| Weather:", df_weather.shape, "| Water monthly:", df_water_m.shape, "| Pop yearly:", df_pop_y.shape)


/tmp/ipykernel_17/3166822607.py:128: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_ndvi["tanggal"] = pd.to_datetime(df_ndvi["tanggal"], errors="coerce", dayfirst=True)


=== STAGE 2 SUMMARY ===
Submission rows: 180
Target date range: 2025-01-09 -> 2025-12-11
Stations in submission: ['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']

Last observed date per station (from ISPU hist):
stasiun_code
DKI1   2023-11-30
DKI2   2023-11-30
DKI3   2023-11-30
DKI4   2023-11-30
DKI5   2023-11-30
Name: tanggal, dtype: datetime64[ns]

Horizon_days summary:
count    180.000000
mean     573.500000
std      105.308406
min      406.000000
25%      488.750000
50%      573.000000
75%      658.000000
max      742.000000
Name: horizon_days, dtype: float64

df_targets preview:
                id tanggal_target stasiun_code last_obs_date  horizon_days  \
0  2025-09-01_DKI1     2025-01-09         DKI1    2023-11-30           406   
1  2025-09-01_DKI2     2025-01-09         DKI2    2023-11-30           406   
2  2025-09-01_DKI3     2025-01-09         DKI3    2023-11-30           406   
3  2025-09-01_DKI4     2025-01-09         DKI4    2023-11-30           406   
4  2025-09-01_DKI5     2025

# Feature Engineering (Time-Series + Calendar + Robustness)

In [4]:
# ============================================================
# STAGE 3 — Feature Engineering (Time-Series + Calendar + Forecasting-safe) — ONE CELL
# Prinsip:
# - Test tidak punya fitur polutan masa depan -> fitur target dibangun dari:
#   (A) "state histori terakhir" per stasiun (rolling/trend/missingness dari df_ispu_hist)
#   (B) kalender/libur pada tanggal_target (sudah ada di df_targets dari Stage 2)
#   (C) horizon_days (+ transform)
#   (D) opsional: baseline musiman berbasis histori (station x month) untuk polutan/NDVI/weather (global, semua <= hist_end)
#
# Output utama:
# - df_targets_feat : df_targets + fitur state/seasonal (siap untuk inference / jadi template training)
# - FEATURE_COLS_TARGET : daftar fitur numerik/kategori yang dipakai (untuk stage 4/5)
# ============================================================

import numpy as np
import pandas as pd

# ---------- guards ----------
need = ["df_ispu_hist", "df_targets", "VALID_STATIONS"]
miss = [k for k in need if k not in globals()]
if miss:
    raise RuntimeError(f"Missing globals from previous stages: {miss}")

# ============
# 3.1 Setup
# ============
POLL_COLS = [c for c in ["pm_sepuluh","pm_duakomalima","sulfur_dioksida","karbon_monoksida","ozon","nitrogen_dioksida","max"] if c in df_ispu_hist.columns]
HIST_END = df_ispu_hist["tanggal"].max()  # histori terakhir yang tersedia (sekarang 2023-11-30)

# pastikan urut
dfh = df_ispu_hist.sort_values(["stasiun_code","tanggal"]).reset_index(drop=True)

# ============
# 3.2 Build rolling/trend features on history (past-only, per stasiun)
# ============
for c in POLL_COLS:
    s = dfh.groupby("stasiun_code")[c]
    dfh[f"{c}_lag1"] = s.shift(1)
    dfh[f"{c}_lag7"] = s.shift(7)

    # rolling pakai jumlah baris (karena data umumnya harian)
    dfh[f"{c}_rmean7"]  = s.shift(1).rolling(7).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd7"]   = s.shift(1).rolling(7).std().reset_index(level=0, drop=True)
    dfh[f"{c}_rmean30"] = s.shift(1).rolling(30).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd30"]  = s.shift(1).rolling(30).std().reset_index(level=0, drop=True)

    # trend pendek vs panjang
    dfh[f"{c}_trend_7_30"] = dfh[f"{c}_rmean7"] - dfh[f"{c}_rmean30"]

    # missingness rate (rolling 30 obs)
    na_s = s.shift(1).isna().astype(float)
    dfh[f"{c}_na_rate30"] = na_s.rolling(30).mean().reset_index(level=0, drop=True)

# ambil snapshot terakhir per stasiun (state pada HIST_END)
df_last = (
    dfh.loc[dfh["tanggal"].eq(HIST_END)]
      .groupby("stasiun_code", as_index=False)
      .tail(1)
      .copy()
)

keep_last = ["stasiun_code"] + [c for c in df_last.columns if any(c.startswith(pc) for pc in POLL_COLS)]
df_station_state = df_last[keep_last].copy()

# ============
# 3.3 Baseline musiman (global) station x month untuk polutan & label prior
#     (dipakai untuk target date; semuanya dibangun dari histori <= HIST_END)
# ============
dfh2 = dfh.copy()
dfh2["month"] = dfh2["tanggal"].dt.month

# polutan baseline per (stasiun, month)
agg_pol = {c: "mean" for c in POLL_COLS}
df_pol_month = (
    dfh2.groupby(["stasiun_code","month"], as_index=False)
        .agg(agg_pol)
)
df_pol_month = df_pol_month.rename(columns={c: f"{c}_mon_mean" for c in POLL_COLS})

# label prior per (stasiun, month) dari baris yang ada label
df_lbl = dfh2.loc[dfh2["kategori_3"].notna(), ["stasiun_code","month","kategori_3"]].copy()
if len(df_lbl) > 0:
    tmp = pd.crosstab([df_lbl["stasiun_code"], df_lbl["month"]], df_lbl["kategori_3"], normalize="index").reset_index()
    # pastikan kolom ada semua
    for k in ["BAIK","SEDANG","TIDAK SEHAT"]:
        if k not in tmp.columns:
            tmp[k] = 0.0
    df_lbl_month = tmp.rename(columns={
        "BAIK": "p_baik_mon",
        "SEDANG": "p_sedang_mon",
        "TIDAK SEHAT": "p_tidak_sehat_mon",
    })
else:
    df_lbl_month = pd.DataFrame(columns=["stasiun_code","month","p_baik_mon","p_sedang_mon","p_tidak_sehat_mon"])

# ============
# 3.4 NDVI features (last-known + monthly baseline) — hanya pakai <= HIST_END
# ============
df_ndvi_use = globals().get("df_ndvi", pd.DataFrame()).copy()
if len(df_ndvi_use) > 0 and {"tanggal","stasiun_code"}.issubset(df_ndvi_use.columns):
    df_ndvi_use = df_ndvi_use.loc[df_ndvi_use["stasiun_code"].isin(VALID_STATIONS)].copy()
    df_ndvi_use["tanggal"] = pd.to_datetime(df_ndvi_use["tanggal"], errors="coerce", dayfirst=True)
    df_ndvi_use = df_ndvi_use.dropna(subset=["tanggal"])
    df_ndvi_use = df_ndvi_use.loc[df_ndvi_use["tanggal"] <= HIST_END].copy()

    ndvi_col = "ndvi" if "ndvi" in df_ndvi_use.columns else None
    if ndvi_col is not None:
        df_ndvi_use[ndvi_col] = pd.to_numeric(df_ndvi_use[ndvi_col], errors="coerce")

        df_ndvi_use["month"] = df_ndvi_use["tanggal"].dt.month
        df_ndvi_last = (
            df_ndvi_use.sort_values(["stasiun_code","tanggal"])
                       .groupby("stasiun_code", as_index=False)
                       .tail(1)[["stasiun_code", ndvi_col]]
                       .rename(columns={ndvi_col: "ndvi_last"})
        )
        df_ndvi_month = (
            df_ndvi_use.groupby(["stasiun_code","month"], as_index=False)[ndvi_col]
                       .mean()
                       .rename(columns={ndvi_col: "ndvi_mon_mean"})
        )
    else:
        df_ndvi_last = pd.DataFrame(columns=["stasiun_code","ndvi_last"])
        df_ndvi_month = pd.DataFrame(columns=["stasiun_code","month","ndvi_mon_mean"])
else:
    df_ndvi_last = pd.DataFrame(columns=["stasiun_code","ndvi_last"])
    df_ndvi_month = pd.DataFrame(columns=["stasiun_code","month","ndvi_mon_mean"])

# ============
# 3.5 Weather monthly baseline (global) — hanya pakai <= HIST_END
#     (bukan "actual future weather"; hanya pola historis)
# ============
df_w = globals().get("df_weather", pd.DataFrame()).copy()
if len(df_w) > 0 and {"tanggal","stasiun_code"}.issubset(df_w.columns):
    df_w = df_w.loc[df_w["stasiun_code"].isin(VALID_STATIONS)].copy()
    df_w["tanggal"] = pd.to_datetime(df_w["tanggal"], errors="coerce", dayfirst=True)
    df_w = df_w.dropna(subset=["tanggal"])
    df_w = df_w.loc[df_w["tanggal"] <= HIST_END].copy()
    df_w["month"] = df_w["tanggal"].dt.month

    # pilih subset kolom cuaca yang umum; ambil yang ada saja
    w_candidates = [
        "temperature_2m_mean", "temperature_2m_max", "temperature_2m_min",
        "precipitation_sum", "precipitation_hours",
        "wind_speed_10m_mean", "wind_speed_10m_max", "wind_speed_10m_min",
        "relative_humidity_2m_mean",
        "surface_pressure_mean",
        "cloud_cover_mean",
        "shortwave_radiation_sum",
        "wind_gusts_10m_mean", "wind_gusts_10m_max"
    ]
    WCOLS = [c for c in w_candidates if c in df_w.columns]
    for c in WCOLS:
        df_w[c] = pd.to_numeric(df_w[c], errors="coerce")

    if len(WCOLS) > 0:
        df_w_month = (
            df_w.groupby(["stasiun_code","month"], as_index=False)[WCOLS]
                .mean()
        )
        df_w_month = df_w_month.rename(columns={c: f"{c}_mon_mean" for c in WCOLS})
    else:
        df_w_month = pd.DataFrame(columns=["stasiun_code","month"])
else:
    df_w_month = pd.DataFrame(columns=["stasiun_code","month"])

# ============
# 3.6 Merge all features into target frame
# ============
df_targets_feat = df_targets.copy()

# horizon transforms
df_targets_feat["horizon_weeks"]  = df_targets_feat["horizon_days"] / 7.0
df_targets_feat["horizon_months"] = df_targets_feat["horizon_days"] / 30.0
df_targets_feat["log1p_horizon"]  = np.log1p(df_targets_feat["horizon_days"].clip(lower=0))

# merge station last state
df_targets_feat = df_targets_feat.merge(df_station_state, on="stasiun_code", how="left")

# merge month baselines (polutan + label priors)
df_targets_feat = df_targets_feat.merge(df_pol_month, on=["stasiun_code","month"], how="left")
df_targets_feat = df_targets_feat.merge(df_lbl_month, on=["stasiun_code","month"], how="left")

# merge NDVI
df_targets_feat = df_targets_feat.merge(df_ndvi_last, on="stasiun_code", how="left")
df_targets_feat = df_targets_feat.merge(df_ndvi_month, on=["stasiun_code","month"], how="left")

# merge weather monthly
df_targets_feat = df_targets_feat.merge(df_w_month, on=["stasiun_code","month"], how="left")

# ============
# 3.7 Define feature columns (untuk stage 4/5)
# ============
# kategori features: stasiun_code, day_name (jika ada), nama_libur (jika ada)
CAT_FEATS = [c for c in ["stasiun_code","day_name","nama_libur"] if c in df_targets_feat.columns]

# numeric features: kalender + horizon + state + baselines + NDVI/weather
BASE_NUM = [c for c in [
    "year","month","day","dow","dayofyear","sin_doy","cos_doy","sin_month","cos_month",
    "is_weekend","is_holiday_nasional",
    "horizon_days","horizon_weeks","horizon_months","log1p_horizon"
] if c in df_targets_feat.columns]

STATE_NUM = [c for c in df_targets_feat.columns if any(c.startswith(pc+"_") for pc in POLL_COLS)]
PRIOR_NUM = [c for c in ["p_baik_mon","p_sedang_mon","p_tidak_sehat_mon","ndvi_last","ndvi_mon_mean"] if c in df_targets_feat.columns]
WEATHER_NUM = [c for c in df_targets_feat.columns if c.endswith("_mon_mean") and c not in [f"{p}_mon_mean" for p in POLL_COLS] and c not in ["ndvi_mon_mean"]]
POLL_MON_NUM = [c for c in df_targets_feat.columns if c.endswith("_mon_mean") and any(c.startswith(p) for p in POLL_COLS)]

NUM_FEATS = BASE_NUM + STATE_NUM + POLL_MON_NUM + PRIOR_NUM + WEATHER_NUM

# dedup lists (jaga urutan)
def _dedup(seq):
    out, seen = [], set()
    for x in seq:
        if x not in seen:
            out.append(x); seen.add(x)
    return out

CAT_FEATS = _dedup(CAT_FEATS)
NUM_FEATS = _dedup(NUM_FEATS)

FEATURE_COLS_TARGET = CAT_FEATS + NUM_FEATS

# ============
# 3.8 Prints (QA)
# ============
print("=== STAGE 3 SUMMARY ===")
print("HIST_END:", HIST_END.date())
print("POLL_COLS:", POLL_COLS)
print("Target feature rows:", len(df_targets_feat))
print("Feature cols:", len(FEATURE_COLS_TARGET), "| cat:", len(CAT_FEATS), "| num:", len(NUM_FEATS))

# cek missing ringan untuk fitur utama
check_cols = ["horizon_days","is_weekend","is_holiday_nasional"] + [c for c in ["max_lag1","max_rmean7","max_rmean30","max_mon_mean"] if c in df_targets_feat.columns]
miss_rate = df_targets_feat[check_cols].isna().mean().sort_values(ascending=False)
print("\nMissing rate (selected features):")
print(miss_rate)

print("\nTargets feature preview:")
show_cols = ["id","tanggal_target","stasiun_code","horizon_days","is_weekend","is_holiday_nasional"]
show_cols += [c for c in ["max_lag1","max_rmean7","max_rmean30","max_mon_mean","p_tidak_sehat_mon","ndvi_last","ndvi_mon_mean"] if c in df_targets_feat.columns]
print(df_targets_feat[show_cols].head(10))


=== STAGE 3 SUMMARY ===
HIST_END: 2023-11-30
POLL_COLS: ['pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max']
Target feature rows: 180
Feature cols: 88 | cat: 3 | num: 85

Missing rate (selected features):
horizon_days           0.0
is_weekend             0.0
is_holiday_nasional    0.0
max_lag1               0.0
max_rmean7             0.0
max_rmean30            0.0
max_mon_mean           0.0
dtype: float64

Targets feature preview:
                id tanggal_target stasiun_code  horizon_days  is_weekend  \
0  2025-09-01_DKI1     2025-01-09         DKI1           406           0   
1  2025-09-01_DKI2     2025-01-09         DKI2           406           0   
2  2025-09-01_DKI3     2025-01-09         DKI3           406           0   
3  2025-09-01_DKI4     2025-01-09         DKI4           406           0   
4  2025-09-01_DKI5     2025-01-09         DKI5           406           0   
5  2025-09-02_DKI1     2025-02-09         DKI1        

# Model Training (Time-Based CV + CatBoost Optimization)

In [5]:
# ============================================================
# STAGE 4 — Model Training (Forecasting-safe + Time-based CV) — ONE CELL (REVISI FULL v3)
# Fix:
# - merge_asof sorted global by cutoff_date ✅
# - sanitize pd.NA -> np.nan (numeric) dan fill "NA" (categorical) ✅
# - cat_features pakai NAMA kolom (bukan index) ✅
# ============================================================

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool

# ---------- guards ----------
need = ["df_ispu_hist", "df_calendar", "df_targets", "VALID_STATIONS"]
miss = [k for k in need if k not in globals()]
if miss:
    raise RuntimeError(f"Missing globals from previous stages: {miss}")

SEED = 42
LABELS = ["BAIK", "SEDANG", "TIDAK SEHAT"]
label_to_id = {k:i for i,k in enumerate(LABELS)}
id_to_label = {i:k for k,i in label_to_id.items()}

# ---------- helpers ----------
def sanitize_features(df: pd.DataFrame, cat_cols, num_cols):
    df = df.copy()
    # categorical: pd.NA -> "NA"
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].astype("string").fillna("NA").astype(str)
    # numeric: pd.NA -> np.nan, cast to float
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")
    return df

# ---------- 4.1 choose training horizons ----------
q_h = df_targets["horizon_days"].quantile([0.0, 0.5, 1.0]).astype(int).tolist()
H_TRAIN = sorted(set([1,3,7,14,30,60,90,180,365,540,730] + q_h))

# ---------- 4.2 build past-only state features on history ----------
dfh = df_ispu_hist.sort_values(["stasiun_code","tanggal"]).reset_index(drop=True)
POLL_COLS = [c for c in ["pm_sepuluh","pm_duakomalima","sulfur_dioksida","karbon_monoksida","ozon","nitrogen_dioksida","max"] if c in dfh.columns]

for c in POLL_COLS:
    g = dfh.groupby("stasiun_code")[c]
    dfh[f"{c}_lag1"] = g.shift(1)
    dfh[f"{c}_lag7"] = g.shift(7)
    dfh[f"{c}_rmean7"]  = g.shift(1).rolling(7).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd7"]   = g.shift(1).rolling(7).std().reset_index(level=0, drop=True)
    dfh[f"{c}_rmean30"] = g.shift(1).rolling(30).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd30"]  = g.shift(1).rolling(30).std().reset_index(level=0, drop=True)
    dfh[f"{c}_trend_7_30"] = dfh[f"{c}_rmean7"] - dfh[f"{c}_rmean30"]
    na_s = g.shift(1).isna().astype(float)
    dfh[f"{c}_na_rate30"] = na_s.rolling(30).mean().reset_index(level=0, drop=True)

state_cols = [c for c in dfh.columns if any(c.startswith(pc+"_") for pc in POLL_COLS)]

df_base = dfh[["stasiun_code","tanggal"] + state_cols].rename(columns={"tanggal":"cutoff_date"}).copy()
df_base["stasiun_code"] = df_base["stasiun_code"].astype("string")
df_base["cutoff_date"] = pd.to_datetime(df_base["cutoff_date"])
df_base = df_base.sort_values(["cutoff_date","stasiun_code"]).reset_index(drop=True)

# ---------- 4.3 label table ----------
df_y = dfh[["stasiun_code","tanggal","kategori_3"]].rename(columns={"tanggal":"target_date"}).copy()
df_y = df_y[df_y["kategori_3"].isin(LABELS)].reset_index(drop=True)
if len(df_y) == 0:
    raise RuntimeError("Tidak ada label kategori_3 valid (BAIK/SEDANG/TIDAK SEHAT) di histori.")

# ---------- 4.4 calendar features for target_date ----------
cal_keep = [c for c in [
    "tanggal","year","month","day","dow","dayofyear","sin_doy","cos_doy","sin_month","cos_month",
    "is_weekend","is_holiday_nasional","day_name","nama_libur"
] if c in df_calendar.columns]
df_cal = df_calendar[cal_keep].rename(columns={"tanggal":"target_date"}).copy()
df_cal["target_date"] = pd.to_datetime(df_cal["target_date"])

# ---------- 4.5 supervised rows for all horizons ----------
parts = []
for h in H_TRAIN:
    d = df_y.copy()
    d["horizon_days"] = int(h)
    d["cutoff_date"] = d["target_date"] - pd.to_timedelta(int(h), unit="D")
    parts.append(d)

df_sup = pd.concat(parts, ignore_index=True)
df_sup = df_sup[df_sup["stasiun_code"].isin(VALID_STATIONS)].copy()

df_sup["stasiun_code"] = df_sup["stasiun_code"].astype("string")
df_sup["target_date"] = pd.to_datetime(df_sup["target_date"])
df_sup["cutoff_date"] = pd.to_datetime(df_sup["cutoff_date"])
df_sup = df_sup.sort_values(["cutoff_date","stasiun_code","target_date","horizon_days"]).reset_index(drop=True)

# merge_asof (need global sort by cutoff_date)
df_sup = pd.merge_asof(
    df_sup, df_base,
    on="cutoff_date", by="stasiun_code",
    direction="backward",
    allow_exact_matches=True,
    tolerance=pd.Timedelta(days=60)
)
df_sup = df_sup.dropna(subset=state_cols).copy()

# merge calendar
df_sup = df_sup.merge(df_cal, on="target_date", how="left")

# horizon transforms
df_sup["horizon_weeks"]  = df_sup["horizon_days"] / 7.0
df_sup["horizon_months"] = df_sup["horizon_days"] / 30.0
df_sup["log1p_horizon"]  = np.log1p(df_sup["horizon_days"].clip(lower=0))

# y
df_sup["y"] = df_sup["kategori_3"].map(label_to_id).astype(int)

# final sort
df_sup = df_sup.sort_values(["target_date","stasiun_code","horizon_days"]).reset_index(drop=True)

print("=== STAGE 4 DATA CHECK ===")
print("H_TRAIN:", H_TRAIN[:8], "...", H_TRAIN[-3:])
print("Supervised rows after merges:", len(df_sup))
print("Target_date range:", df_sup["target_date"].min().date(), "->", df_sup["target_date"].max().date())
years = sorted(df_sup["target_date"].dt.year.unique().tolist())
print("Unique target years:", years)

# ---------- 4.6 feature columns ----------
CAT_FEATS_MODEL = [c for c in ["stasiun_code","day_name","nama_libur"] if c in df_sup.columns]

BASE_NUM = [c for c in [
    "year","month","day","dow","dayofyear","sin_doy","cos_doy","sin_month","cos_month",
    "is_weekend","is_holiday_nasional",
    "horizon_days","horizon_weeks","horizon_months","log1p_horizon"
] if c in df_sup.columns]
NUM_FEATS_MODEL = BASE_NUM + state_cols

def _dedup(seq):
    out, seen = [], set()
    for x in seq:
        if x not in seen:
            out.append(x); seen.add(x)
    return out

CAT_FEATS_MODEL = _dedup(CAT_FEATS_MODEL)
NUM_FEATS_MODEL = _dedup(NUM_FEATS_MODEL)
FEATURE_COLS_MODEL = CAT_FEATS_MODEL + NUM_FEATS_MODEL

# sanitize entire table once (prevents pd.NA leaking to CatBoost)
df_sup = sanitize_features(df_sup, CAT_FEATS_MODEL, NUM_FEATS_MODEL)

# ---------- 4.7 folds (time-based, fallback) ----------
folds = []
if len(years) >= 3:
    for vy in years[-3:]:
        tr_mask = df_sup["target_date"].dt.year < vy
        va_mask = df_sup["target_date"].dt.year == vy
        folds.append((f"val_year_{vy}", tr_mask, va_mask))
else:
    dates = df_sup["target_date"].sort_values().unique()
    split_a = dates[int(len(dates)*0.8)]
    split_b = dates[int(len(dates)*0.9)]
    folds = [
        ("time_split_1", df_sup["target_date"] < split_a, df_sup["target_date"] >= split_a),
        ("time_split_2", df_sup["target_date"] < split_b, (df_sup["target_date"] >= split_a) & (df_sup["target_date"] < split_b)),
    ]

# ---------- 4.8 class weights ----------
counts = df_sup["y"].value_counts().to_dict()
tot = len(df_sup)
class_weights = {k: (tot / (len(LABELS) * counts.get(k, 1))) for k in range(len(LABELS))}

# ---------- 4.9 train folds + OOF ----------
oof_pred_proba = np.full((len(df_sup), len(LABELS)), np.nan, dtype=float)
models = {}
cv_rows = []

for name, tr_mask, va_mask in folds:
    ntr = int(tr_mask.sum())
    nva = int(va_mask.sum())
    if ntr == 0 or nva == 0:
        cv_rows.append({"fold": name, "n_train": ntr, "n_valid": nva, "macro_f1": np.nan})
        continue

    X_tr = df_sup.loc[tr_mask, FEATURE_COLS_MODEL]
    y_tr = df_sup.loc[tr_mask, "y"].values
    X_va = df_sup.loc[va_mask, FEATURE_COLS_MODEL]
    y_va = df_sup.loc[va_mask, "y"].values

    tr_pool = Pool(X_tr, label=y_tr, cat_features=CAT_FEATS_MODEL)
    va_pool = Pool(X_va, label=y_va, cat_features=CAT_FEATS_MODEL)

    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="MultiClass",
        iterations=1200,
        learning_rate=0.06,
        depth=6,
        l2_leaf_reg=3.0,
        random_seed=SEED,
        class_weights=class_weights,
        od_type="Iter",
        od_wait=150,
        verbose=200
    )
    model.fit(tr_pool, eval_set=va_pool, use_best_model=True)

    proba = model.predict_proba(X_va)
    oof_pred_proba[va_mask.values] = proba
    pred = proba.argmax(axis=1)
    f1m = float(f1_score(y_va, pred, average="macro"))

    models[name] = model
    cv_rows.append({"fold": name, "n_train": ntr, "n_valid": nva, "macro_f1": f1m})

cv_report = pd.DataFrame(cv_rows)

oof_mask = ~np.isnan(oof_pred_proba).any(axis=1)
oof_true = df_sup.loc[oof_mask, "y"].values
oof_pred = oof_pred_proba[oof_mask].argmax(axis=1)
oof_macro_f1 = float(f1_score(oof_true, oof_pred, average="macro")) if len(oof_true) else np.nan

print("\n=== STAGE 4 SUMMARY ===")
print("Supervised rows used:", len(df_sup))
print("Features:", len(FEATURE_COLS_MODEL), "| cat:", len(CAT_FEATS_MODEL), "| num:", len(NUM_FEATS_MODEL))
print("\nCV report:")
print(cv_report)
print("\nOOF macro F1:", oof_macro_f1)

# expose globals for Stage 5
df_train_sup = df_sup
FEATURE_COLS_MODEL_USED = FEATURE_COLS_MODEL
CAT_FEATS_MODEL_USED = CAT_FEATS_MODEL
models_by_fold = models
oof_true_label = pd.Series(oof_true).map(id_to_label).values if len(oof_true) else np.array([])
oof_pred_label = pd.Series(oof_pred).map(id_to_label).values if len(oof_true) else np.array([])


=== STAGE 4 DATA CHECK ===
H_TRAIN: [1, 3, 7, 14, 30, 60, 90, 180] ... [573, 730, 742]
Supervised rows after merges: 5025
Target_date range: 2022-12-01 -> 2023-11-30
Unique target years: [2022, 2023]
0:	learn: 1.0462247	test: 1.0894977	best: 1.0894977 (0)	total: 101ms	remaining: 2m 1s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 1.061064755
bestIteration = 2

Shrink model to first 3 iterations.
0:	learn: 1.0504508	test: 1.0613777	best: 1.0613777 (0)	total: 26.1ms	remaining: 31.3s
200:	learn: 0.2620801	test: 0.2740005	best: 0.2740005 (200)	total: 5.82s	remaining: 28.9s
400:	learn: 0.1724186	test: 0.1617870	best: 0.1617592 (399)	total: 11.6s	remaining: 23.1s
600:	learn: 0.1338823	test: 0.1214979	best: 0.1214979 (600)	total: 17.4s	remaining: 17.3s
800:	learn: 0.1097142	test: 0.1018321	best: 0.1018321 (800)	total: 23.1s	remaining: 11.5s
1000:	learn: 0.0924185	test: 0.0875848	best: 0.0875783 (993)	total: 29s	remaining: 5.77s
1199:	learn: 0.0807649	test: 0.0778654	best:

# Inference, Ensembling, Submission & QA

In [6]:
# ============================================================
# STAGE 5 — Inference (Rebuild from sample_submission), Ensemble, Submission & QA — ONE CELL (REVISI FULL)
# - Parse tanggal dari ID pakai format YYYY-MM-DD (AMAN)
# - Build fitur inference sesuai FEATURE_COLS_MODEL_USED (Stage 4)
# - Ensemble predict_proba dari models_by_fold
# - Output /kaggle/working/submission.csv + QA
# ============================================================

import re, json
import numpy as np
import pandas as pd
from pathlib import Path

# ---------- guards ----------
need = ["df_ispu_hist", "models_by_fold", "FEATURE_COLS_MODEL_USED", "CAT_FEATS_MODEL_USED", "VALID_STATIONS"]
miss = [k for k in need if k not in globals()]
if miss:
    raise RuntimeError(f"Missing globals from previous stages: {miss}")

LABELS = ["BAIK", "SEDANG", "TIDAK SEHAT"]
id_to_label = {i:k for i,k in enumerate(LABELS)}

DATA_ROOT = Path("/kaggle/input/penyisihan-datavidia-10")
SUB_PATH  = DATA_ROOT / "sample_submission.csv"
HOL_PATH  = DATA_ROOT / "libur-nasional" / "dataset-libur-nasional-dan-weekend.csv"

OUT_SUB = Path("/kaggle/working/submission.csv")
OUT_QA  = Path("/kaggle/working/qa_submission.json")

FEATURE_COLS = FEATURE_COLS_MODEL_USED
CAT_COLS = CAT_FEATS_MODEL_USED
NUM_COLS = [c for c in FEATURE_COLS if c not in CAT_COLS]

def sanitize_features(df: pd.DataFrame, cat_cols, num_cols):
    df = df.copy()
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].astype("string").fillna("NA").astype(str)
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")
    return df

def extract_station(id_series: pd.Series) -> pd.Series:
    x = id_series.astype(str).str.upper()
    code = x.str.extract(r"(DKI\s*\d+)", expand=False)
    return code.str.replace(" ", "", regex=False)

def extract_date_ymd(id_series: pd.Series) -> pd.Series:
    # ambil token YYYY-MM-DD dari id, parse FIXED format
    tok = id_series.astype(str).str.extract(r"(\d{4}-\d{2}-\d{2})", expand=False)
    return pd.to_datetime(tok, format="%Y-%m-%d", errors="coerce")

# ============================================================
# 5.1 Read sample_submission & parse id
# ============================================================
df_sample = pd.read_csv(SUB_PATH)
sample_cols = [c.strip() for c in df_sample.columns.tolist()]
id_col = sample_cols[0]
target_col = sample_cols[1] if len(sample_cols) > 1 else "kategori"

df_pred = pd.DataFrame({id_col: df_sample[id_col].astype(str)})
df_pred["target_date"] = extract_date_ymd(df_pred[id_col])
df_pred["stasiun_code"] = extract_station(df_pred[id_col])

df_pred = df_pred[df_pred["target_date"].notna() & df_pred["stasiun_code"].notna()].copy()
df_pred = df_pred[df_pred["stasiun_code"].isin(VALID_STATIONS)].reset_index(drop=True)

# ============================================================
# 5.2 last_obs_date & horizon_days (cutoff efektif = data terakhir yang tersedia)
# ============================================================
dfh = df_ispu_hist.sort_values(["stasiun_code","tanggal"]).reset_index(drop=True)
last_obs_by_station = dfh.groupby("stasiun_code")["tanggal"].max()
df_pred["last_obs_date"] = df_pred["stasiun_code"].map(last_obs_by_station)
df_pred["horizon_days"] = (df_pred["target_date"] - df_pred["last_obs_date"]).dt.days.astype(int)
df_pred["cutoff_date"] = df_pred["last_obs_date"]

# ============================================================
# 5.3 Build state features table (same as Stage 4) then merge to cutoff_date
# ============================================================
POLL_COLS = [c for c in ["pm_sepuluh","pm_duakomalima","sulfur_dioksida","karbon_monoksida","ozon","nitrogen_dioksida","max"] if c in dfh.columns]

for c in POLL_COLS:
    g = dfh.groupby("stasiun_code")[c]
    dfh[f"{c}_lag1"] = g.shift(1)
    dfh[f"{c}_lag7"] = g.shift(7)
    dfh[f"{c}_rmean7"]  = g.shift(1).rolling(7).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd7"]   = g.shift(1).rolling(7).std().reset_index(level=0, drop=True)
    dfh[f"{c}_rmean30"] = g.shift(1).rolling(30).mean().reset_index(level=0, drop=True)
    dfh[f"{c}_rstd30"]  = g.shift(1).rolling(30).std().reset_index(level=0, drop=True)
    dfh[f"{c}_trend_7_30"] = dfh[f"{c}_rmean7"] - dfh[f"{c}_rmean30"]
    dfh[f"{c}_na_rate30"]  = g.shift(1).isna().astype(float).rolling(30).mean().reset_index(level=0, drop=True)

state_cols = [c for c in dfh.columns if any(c.startswith(pc+"_") for pc in POLL_COLS)]
df_base = dfh[["stasiun_code","tanggal"] + state_cols].rename(columns={"tanggal":"cutoff_date"}).copy()
df_base["stasiun_code"] = df_base["stasiun_code"].astype("string")
df_base["cutoff_date"] = pd.to_datetime(df_base["cutoff_date"])
df_base = df_base.sort_values(["cutoff_date","stasiun_code"]).reset_index(drop=True)

df_pred["stasiun_code"] = df_pred["stasiun_code"].astype("string")
df_pred["cutoff_date"] = pd.to_datetime(df_pred["cutoff_date"])
df_pred = df_pred.sort_values(["cutoff_date","stasiun_code"]).reset_index(drop=True)

df_pred = pd.merge_asof(
    df_pred, df_base,
    on="cutoff_date", by="stasiun_code",
    direction="backward",
    allow_exact_matches=True,
    tolerance=pd.Timedelta(days=60)
)

# ============================================================
# 5.4 Calendar + holiday features on target_date (recompute directly, safe)
# ============================================================
df_pred["year"] = df_pred["target_date"].dt.year
df_pred["month"] = df_pred["target_date"].dt.month
df_pred["day"] = df_pred["target_date"].dt.day
df_pred["dow"] = df_pred["target_date"].dt.dayofweek
df_pred["dayofyear"] = df_pred["target_date"].dt.dayofyear
df_pred["sin_doy"] = np.sin(2*np.pi*df_pred["dayofyear"]/365.25)
df_pred["cos_doy"] = np.cos(2*np.pi*df_pred["dayofyear"]/365.25)
df_pred["sin_month"] = np.sin(2*np.pi*df_pred["month"]/12.0)
df_pred["cos_month"] = np.cos(2*np.pi*df_pred["month"]/12.0)
df_pred["is_weekend"] = (df_pred["dow"] >= 5).astype(int)

df_hol = pd.read_csv(HOL_PATH)
df_hol.columns = [str(c).strip().lower() for c in df_hol.columns]
df_hol["tanggal"] = pd.to_datetime(df_hol["tanggal"], errors="coerce")
keep_h = [c for c in ["tanggal","is_holiday_nasional","nama_libur","day_name"] if c in df_hol.columns]
df_hol = df_hol[keep_h].dropna(subset=["tanggal"]).drop_duplicates("tanggal")

df_pred = df_pred.merge(df_hol, left_on="target_date", right_on="tanggal", how="left").drop(columns=["tanggal"], errors="ignore")
df_pred["is_holiday_nasional"] = pd.to_numeric(df_pred.get("is_holiday_nasional", 0), errors="coerce").fillna(0).astype(int)

# horizon transforms
df_pred["horizon_weeks"] = df_pred["horizon_days"] / 7.0
df_pred["horizon_months"] = df_pred["horizon_days"] / 30.0
df_pred["log1p_horizon"] = np.log1p(df_pred["horizon_days"].clip(lower=0))

# ============================================================
# 5.5 Build X with exact FEATURE_COLS_MODEL_USED (add missing cols if any)
# ============================================================
X = df_pred.copy()
for c in FEATURE_COLS:
    if c not in X.columns:
        X[c] = np.nan

X = sanitize_features(X, CAT_COLS, NUM_COLS)

# ============================================================
# 5.6 Ensemble predict_proba
# ============================================================
models = list(models_by_fold.values())
if len(models) == 0:
    raise RuntimeError("models_by_fold kosong. Pastikan Stage 4 sukses.")

proba_sum = None
for m in models:
    p = np.asarray(m.predict_proba(X[FEATURE_COLS]), dtype=float)
    proba_sum = p if proba_sum is None else (proba_sum + p)

proba_mean = proba_sum / len(models)
pred = proba_mean.argmax(axis=1)
pred_label = pd.Series(pred).map(id_to_label).values

pred_df = pd.DataFrame({id_col: df_pred[id_col].values, target_col: pred_label})

# ============================================================
# 5.7 Build final submission in EXACT sample order (NO KeyError)
# ============================================================
sub = df_sample[[id_col]].merge(pred_df, on=id_col, how="left")

# fallback if any missing predictions (should be 0)
sub[target_col] = sub[target_col].fillna("SEDANG")

sub.to_csv(OUT_SUB, index=False)

# ============================================================
# 5.8 QA
# ============================================================
qa = {}
qa["rows_sample_submission"] = int(len(df_sample))
qa["rows_pred_parsed"] = int(len(df_pred))
qa["rows_submission"] = int(len(sub))
qa["missing_pred_after_merge"] = int(sub[target_col].isna().sum())
qa["label_set_ok"] = bool(set(sub[target_col].unique()).issubset(set(LABELS)))
qa["pred_distribution"] = {str(k): int(v) for k,v in sub[target_col].value_counts(dropna=False).to_dict().items()}
qa["n_models_ensembled"] = int(len(models))
qa["proba_min"] = float(np.nanmin(proba_mean))
qa["proba_max"] = float(np.nanmax(proba_mean))

OUT_QA.write_text(json.dumps(qa, indent=2))

print("=== STAGE 5 SUMMARY ===")
print("[OK] submission:", str(OUT_SUB))
print("[OK] qa:", str(OUT_QA))
print("QA:", qa)
print("\nSubmission head:")
print(sub.head(10))


=== STAGE 5 SUMMARY ===
[OK] submission: /kaggle/working/submission.csv
[OK] qa: /kaggle/working/qa_submission.json
QA: {'rows_sample_submission': 455, 'rows_pred_parsed': 455, 'rows_submission': 455, 'missing_pred_after_merge': 0, 'label_set_ok': True, 'pred_distribution': {'SEDANG': 382, 'TIDAK SEHAT': 73}, 'n_models_ensembled': 2, 'proba_min': 0.13974515556888395, 'proba_max': 0.6875149061710245}

Submission head:
                id     category
0  2025-09-01_DKI1  TIDAK SEHAT
1  2025-09-01_DKI2  TIDAK SEHAT
2  2025-09-01_DKI3       SEDANG
3  2025-09-01_DKI4  TIDAK SEHAT
4  2025-09-01_DKI5       SEDANG
5  2025-09-02_DKI1  TIDAK SEHAT
6  2025-09-02_DKI2  TIDAK SEHAT
7  2025-09-02_DKI3       SEDANG
8  2025-09-02_DKI4  TIDAK SEHAT
9  2025-09-02_DKI5       SEDANG
